# The Reflection Pattern

This chapter reveals a key insight: **AI agents are just automated loops**.

We'll implement the same task two ways — showing how the agent automates what you can build manually.

---

## What is Reflection?

The **Reflection pattern** is a loop where the AI:
1. **Generates** an output
2. **Critiques** its own output
3. **Refines** based on the critique
4. **Repeats** until satisfied

```
┌─────────────────────────────────────────────────────────────────────┐
│                        REFLECTION LOOP                              │
│                                                                     │
│    ┌──────────┐      ┌──────────┐      ┌──────────┐                │
│    │ GENERATE │─────▶│ CRITIQUE │─────▶│  REFINE  │                │
│    │          │      │          │      │          │                │
│    │  Draft   │      │  Check   │      │   Fix    │                │
│    │  output  │      │  issues  │      │  issues  │                │
│    └──────────┘      └────┬─────┘      └────┬─────┘                │
│                           │                 │                       │
│                           │ Issues?         │                       │
│                           │                 │                       │
│                      No ──┴── Yes ──────────┘                       │
│                           │                 ▲                       │
│                           ▼                 │                       │
│                    ┌──────────┐             │                       │
│                    │  OUTPUT  │      (repeat if needed)            │
│                    └──────────┘                                     │
└─────────────────────────────────────────────────────────────────────┘
```

This pattern is validated by research — the **Self-Refine paper** (Madaan et al., 2023) shows it improves task performance by ~20% on average.

**How is this different from Prompt Chaining?** Prompt chaining is predefined steps that always run the same way. Reflection is *adaptive iteration based on feedback* — the loop continues until the output meets criteria.

**Key insight:** This is exactly what AI agents do internally. The agent node just automates this loop for you.

---

## The Challenge: Constrained Product Description

To demonstrate reflection, we need a task that LLMs typically fail on the first try. Writing with **multiple hard constraints** is perfect:

| Constraint | Requirement |
|------------|-------------|
| **Sentences** | Exactly 3 |
| **Words** | 25–30 total |
| **Keyword** | Contains "sound" exactly twice |
| **Forbidden** | Does NOT contain "music" or "audio" |

**Product:** Wireless Bluetooth Headphones

Try asking ChatGPT to do this in one shot — it almost always fails at least one constraint. This is why reflection matters.

---

## Two Approaches, One Pattern

We'll implement the same reflection loop two ways:

| Version | Who controls the loop? | Best for |
|---------|------------------------|----------|
| **V1: Loop with Exit** | You design the loop | When you need full control |
| **V2: AI Agent** | AI decides when to stop | When you want simplicity |

> **Import via URL** (copy and paste in n8n → Import from URL):
> ```
> https://raw.githubusercontent.com/ezponda/ai-agents-course/main/courses/n8n_no_code/book/_static/workflows/05_reflection_pattern.json
> ```
>
> **Download:** {download}`05_reflection_pattern.json <_static/workflows/05_reflection_pattern.json>`

Both versions are in the same workflow. Each has its own trigger — run them separately and compare!

---

## Version 1: Loop with Exit Condition

You design a workflow that **loops until the constraints pass** (or hits a maximum of 5 iterations).

```
┌──────────────────────────────────────────────────────────────────────────────────────┐
│                                    LOOP                                              │
│  ┌───────────────┐   ┌───────────────┐   ┌───────────────┐   ┌───────────────┐      │
│  │ Generate/     │──▶│   Validate    │──▶│ Parse Result  │──▶│   If Pass?    │──┬──▶│ Output
│  │ Refine (LLM)  │   │    (LLM)      │   │    (Set)      │   │               │  │   │
│  └───────────────┘   └───────────────┘   └───────────────┘   └───────────────┘  │   │
│         ▲                                                            │ No       │   │
│         │                                                            ▼          │   │
│         │                                                    ┌───────────────┐  │   │
│         └────────────────────────────────────────────────────│  If < Max?    │──┘   │
│                              (loop back)                     └───────────────┘      │
└──────────────────────────────────────────────────────────────────────────────────────┘
```

**Pros:** Full control, predictable behavior, easy to debug
**Cons:** More complex to build

::::{dropdown} 🛠️ Build V1 (Loop) from scratch (step-by-step)
:color: secondary

### Step 1: Create a new workflow

1. Click **Workflows** → **Add Workflow**
2. Rename it to "Reflection Pattern — V1 Loop"

### Step 2: Add the trigger and input

1. Add **Manual Trigger** → rename to `Run: V1 Loop`
2. Add **Edit Fields** → rename to `Input — Product (V1)`
3. Add four fields:

| Name | Type | Value |
|------|------|-------|
| `product` | String | `Wireless Bluetooth Headphones` |
| `constraints` | String | `- Exactly 3 sentences`<br>`- 25–30 words total`<br>`- Contains the word "sound" exactly twice`<br>`- Does NOT contain the words "music" or "audio"` |
| `iteration` | Number | `0` |
| `draft` | String | (leave empty) |

### Step 3: Add Generate/Refine (Basic LLM Chain)

1. Add **Basic LLM Chain** → rename to `Generate/Refine (V1)`
2. Configure:
   - **Source for Prompt**: `Define below`
   - **Prompt** (Expression mode):
     ```
     Product: {{ $node['Input — Product (V1)'].json.product }}

     Constraints:
     {{ $json.constraints }}

     {{ $json.draft ? 'Previous draft:\n' + $json.draft + '\n\nImprove the draft to fix the issues.' : 'Write the product description:' }}
     ```
3. Add System Message:
   ```
   You are a product copywriter.

   Write a product description that meets ALL the constraints exactly.
   Count words carefully. Output ONLY the description.
   ```
4. Add your Chat Model (OpenRouter, OpenAI, etc.)

### Step 4: Add Store Draft (Edit Fields)

1. Add **Edit Fields** → rename to `Store Draft (V1)`
2. Add three fields:

| Name | Type | Value (Expression) |
|------|------|---------------------|
| `draft` | String | `{{ $json.text }}` |
| `constraints` | String | `{{ $node['Input — Product (V1)'].json.constraints }}` |
| `iteration` | Number | `{{ ($node['Input — Product (V1)'].json.iteration || 0) + 1 }}` |

### Step 5: Add Validate (Basic LLM Chain)

1. Add **Basic LLM Chain** → rename to `Validate (V1)`
2. Configure:
   - **Source for Prompt**: `Define below`
   - **Prompt** (Expression):
     ```
     Draft:
     {{ $json.draft }}

     Constraints:
     {{ $json.constraints }}

     Validation report:
     ```
3. Add System Message:
   ```
   You are a strict validator.

   Check the draft against EACH constraint:
   1. Count sentences (must be exactly 3)
   2. Count words (must be 25–30)
   3. Count occurrences of "sound" (must be exactly 2)
   4. Check for forbidden words "music" or "audio" (must have 0)

   Return a JSON object:
   {
     "sentences": <number>,
     "words": <number>,
     "sound_count": <number>,
     "has_forbidden": <true/false>,
     "all_pass": <true/false>,
     "issues": ["list of issues if any"]
   }

   Return ONLY the JSON.
   ```
4. Connect the same Chat Model

### Step 6: Add Parse Validation (Edit Fields)

1. Add **Edit Fields** → rename to `Parse Validation (V1)`
2. Add five fields:

| Name | Type | Value (Expression) |
|------|------|---------------------|
| `draft` | String | `{{ $node['Store Draft (V1)'].json.draft }}` |
| `constraints` | String | `{{ $node['Store Draft (V1)'].json.constraints }}` |
| `iteration` | Number | `{{ $node['Store Draft (V1)'].json.iteration }}` |
| `validation_json` | String | `{{ $json.text }}` |
| `all_pass` | Boolean | `{{ $json.text.includes('"all_pass": true') || $json.text.includes('"all_pass":true') }}` |

### Step 7: Add If Pass? (If node)

1. Add **If** → rename to `If Pass?`
2. Configure the condition:
   - **Value 1**: `{{ $json.all_pass }}` (Expression)
   - **Operation**: `equals`
   - **Value 2**: `true` (Boolean)

### Step 8: Add Output (V1) — True branch

1. From the **True** output of If Pass?, add **Edit Fields** → rename to `Output (V1)`
2. Add three fields:

| Name | Type | Value (Expression) |
|------|------|---------------------|
| `final_description` | String | `{{ $json.draft }}` |
| `iterations` | Number | `{{ $json.iteration }}` |
| `approach` | String | `V1: Loop with Exit Condition` |

3. Enable **Keep Only Set**

### Step 9: Add If < Max? (If node) — False branch

1. From the **False** output of If Pass?, add **If** → rename to `If < Max?`
2. Configure:
   - **Value 1**: `{{ $json.iteration }}` (Expression)
   - **Operation**: `smaller` (less than)
   - **Value 2**: `5` (Number)

### Step 10: Add Prepare Refine (Edit Fields) — loop back

1. From the **True** output of If < Max?, add **Edit Fields** → rename to `Prepare Refine (V1)`
2. Add three fields:

| Name | Type | Value (Expression) |
|------|------|---------------------|
| `draft` | String | `{{ $json.draft }}` |
| `constraints` | String | `{{ $json.constraints }}` |
| `iteration` | Number | `{{ $json.iteration }}` |

3. **Connect the output of Prepare Refine back to Generate/Refine (V1)** — this creates the loop!

### Step 11: Add Output Max (V1) — guardrail exit

1. From the **False** output of If < Max?, add **Edit Fields** → rename to `Output Max (V1)`
2. Add three fields:

| Name | Type | Value (Expression) |
|------|------|---------------------|
| `final_description` | String | `{{ $json.draft }}` |
| `iterations` | Number | `{{ $json.iteration }}` |
| `approach` | String | `V1: Loop (max iterations reached)` |

3. Enable **Keep Only Set**

### Step 12: Test

1. Click **Execute Workflow**
2. Watch the loop iterate until constraints pass (or max 5)
3. Check how many iterations it took

::::

### Meet the Node: If (Conditional)

The **If** node checks a condition and sends data to one of two outputs.

| Property | Description |
|----------|-------------|
| **Purpose** | Route data based on a condition |
| **Outputs** | True branch (top) and False branch (bottom) |
| **Common use** | Exit loops, handle errors, route by value |

In V1, we use two If nodes:
- **If Pass?** — checks if `all_pass` is true (exit the loop)
- **If < Max?** — checks if `iteration < 5` (prevents infinite loops)

### Parsing the Validation Result

The Validate LLM returns JSON like `{"all_pass": true, ...}`. We need to extract the `all_pass` value for the If node.

Instead of complex JSON parsing, we use a simple **string check** in an expression:

```
{{ $json.text.includes('"all_pass": true') || $json.text.includes('"all_pass":true') }}
```

This returns `true` if the LLM said all constraints passed, `false` otherwise.

**Why this works:**
- The LLM is instructed to output ONLY JSON
- We just need to know if `all_pass` is `true` — no need to parse the full object
- String matching is simpler and more reliable than JSON parsing in expressions

### How the Loop Works

The key to looping in n8n: **connect the output of a node back to an earlier node**.

```
Generate/Refine ◄──────────────────────────────────┐
     │                                             │
     ▼                                             │
  Validate                                         │
     │                                             │
     ▼                                             │
  Parse Result (Set)                               │
     │                                             │
     ▼                                             │
  If Pass? ───Yes──▶ Output                        │
     │                                             │
     No                                            │
     │                                             │
     ▼                                             │
  If < Max? ───Yes──▶ Prepare Refine ──────────────┘
     │
     No (guardrail: max 5 iterations)
     │
     ▼
  Output (max reached)
```

**Important:** The "If < Max?" node is a **guardrail** that prevents infinite loops. Always include this when building loops!

### Node-by-Node Walkthrough (V1)

| Node | Type | What it does |
|------|------|-------------|
| **Run: V1 Loop** | Manual Trigger | Starts Version 1 |
| **Input — Product (V1)** | Set | Creates `product`, `constraints`, `iteration=0`, `draft=""` |
| **Generate/Refine (V1)** | Basic LLM Chain | Generates or refines based on whether `draft` is empty |
| **Store Draft (V1)** | Set | Saves draft and increments iteration |
| **Validate (V1)** | Basic LLM Chain | Checks constraints, returns JSON |
| **Parse Validation (V1)** | Set | Extracts `all_pass` boolean using string check |
| **If Pass?** | If | Routes to Output if `all_pass`, else continues |
| **If < Max?** | If | Guardrail: checks if iteration < 5 |
| **Prepare Refine (V1)** | Set | Formats data for next iteration |
| **Output (V1)** | Set | Returns `final_description`, `iterations`, `approach` |
| **Output Max (V1)** | Set | Returns result when max iterations reached |

---

## Version 2: AI Agent

The simplest to build: let the agent handle the loop.

```
┌──────────────────────────────────────────────────────────────────────────────┐
│                              AI AGENT                                        │
│                                                                              │
│    ┌─────────────────────────────────────────────────────────────────┐      │
│    │                    INTERNAL LOOP                                │      │
│    │                                                                 │      │
│    │   ┌──────────┐      ┌──────────────┐      ┌──────────┐         │      │
│    │   │  THINK   │─────▶│ Call Tool:   │─────▶│  CHECK   │         │      │
│    │   │          │      │  Validator   │      │          │         │      │
│    │   │ "Draft   │      │  (Python)    │      │ "all_pass │──No────┤      │
│    │   │  a desc" │      │              │      │  true?"  │         │      │
│    │   └──────────┘      └──────────────┘      └────┬─────┘         │      │
│    │         ▲                                      │               │      │
│    │         │                                      │ Yes           │      │
│    │         └──────────────────────────────────────┘               │      │
│    │                                                                 │      │
│    └─────────────────────────────────────────────────────────────────┘      │
│                                                                              │
│    ┌─────────────┐                                                          │
│    │  Validator  │ ◄── Python tool that checks constraints                  │
│    │    Tool     │                                                          │
│    └─────────────┘                                                          │
└──────────────────────────────────────────────────────────────────────────────┘
```

**Pros:** Simplest to build, agent adapts strategy
**Cons:** Less predictable, may take more iterations

::::{dropdown} 🛠️ Build V2 (Agent) from scratch (step-by-step)
:color: secondary

### Step 1: Create a new workflow

1. Click **Workflows** → **Add Workflow**
2. Rename it to "Reflection Pattern — V2 Agent"

### Step 2: Add the trigger and input

1. Add **Manual Trigger** → rename to `Run: V2 Agent`
2. Add **Edit Fields** → rename to `Input — Product (V2)`
3. Add two fields:

| Name | Type | Value |
|------|------|-------|
| `product` | String | `Wireless Bluetooth Headphones` |
| `constraints` | String | `- Exactly 3 sentences`<br>`- 25–30 words total`<br>`- Contains the word "sound" exactly twice`<br>`- Does NOT contain the words "music" or "audio"` |

### Step 3: Add the AI Agent

1. Add **AI Agent** → rename to `Reflection Agent (V2)`
2. Configure:
   - **Source for Prompt**: `Define below`
   - **Prompt** (Expression mode):
     ```
     Product: {{ $json.product }}

     Constraints:
     {{ $json.constraints }}

     Write a product description that meets ALL constraints. After writing, use the Validator tool to check it. Keep refining until all constraints pass.
     ```
3. Click **Options** → Add **System Message**:
   ```
   You are a product copywriter who writes precise descriptions.

   Process:
   1. Write a draft description
   2. Use the Validator tool to check ALL constraints
   3. If any constraint fails, revise and validate again
   4. Repeat until all_pass is true
   5. Return the final description

   Rules:
   - Count words carefully
   - Call the Validator tool at most 5 times
   - When all_pass is true, return ONLY the final description
   ```

### Step 4: Add the Chat Model (sub-node)

1. Click **+ Chat Model** at the bottom of the Agent node
2. Select **OpenRouter Chat Model** (or OpenAI, Google, etc.)
3. Choose your credential
4. Model: `openai/gpt-4o-mini` (or similar)

### Step 5: Add the Validator Tool (Python Code Tool)

1. Click **+ Tool** at the bottom of the Agent node
2. Select **Code Tool**
3. Configure:
   - **Name**: `Validator`
   - **Description**: `Validates a product description against constraints. Input: the description text. Returns JSON with validation results including all_pass (true/false).`
   - **Language**: `Python`
   - **Python Code**:
     ```python
     import re

     # Get the description from the agent's input
     description = _input.first().json.get('query', '') or _input.first().json.get('input', '')

     # Count sentences (split by . ! ?)
     sentences = [s for s in re.split(r'[.!?]+', description) if s.strip()]
     sentence_count = len(sentences)

     # Count words
     words = [w for w in description.split() if w]
     word_count = len(words)

     # Count "sound" occurrences (case insensitive)
     sound_matches = re.findall(r'\\bsound\\b', description.lower())
     sound_count = len(sound_matches)

     # Check for forbidden words
     has_forbidden = bool(re.search(r'\\b(music|audio)\\b', description, re.IGNORECASE))

     # Check all constraints
     issues = []
     if sentence_count != 3:
         issues.append(f'Sentences: {sentence_count} (need exactly 3)')
     if word_count < 25 or word_count > 30:
         issues.append(f'Words: {word_count} (need 25-30)')
     if sound_count != 2:
         issues.append(f'"sound" count: {sound_count} (need exactly 2)')
     if has_forbidden:
         issues.append('Contains forbidden word: "music" or "audio"')

     all_pass = len(issues) == 0

     return [{
         'json': {
             'sentences': sentence_count,
             'words': word_count,
             'sound_count': sound_count,
             'has_forbidden': has_forbidden,
             'all_pass': all_pass,
             'issues': issues
         }
     }]
     ```

### Step 6: Add Output (Edit Fields)

1. Add **Edit Fields** → rename to `Output (V2)`
2. Add two fields:

| Name | Type | Value (Expression) |
|------|------|---------------------|
| `final_description` | String | `{{ $json.output }}` |
| `approach` | String | `V2: AI Agent with Validator Tool` |

3. Enable **Keep Only Set**

### Step 7: Test

1. Click **Execute Workflow**
2. Click the **Agent node** to see its internal tool calls
3. Watch how it writes, validates, and refines automatically

::::

### The Validator Tool (Python)

Instead of using an LLM to validate, V2 uses a **Python Code tool** — a function the agent can call:

```python
import re

# Get the description from the agent's input
description = _input.first().json.get('query', '') or _input.first().json.get('input', '')

# Count sentences (split by . ! ?)
sentences = [s for s in re.split(r'[.!?]+', description) if s.strip()]
sentence_count = len(sentences)

# Count words
words = [w for w in description.split() if w]
word_count = len(words)

# Count "sound" occurrences (case insensitive)
sound_matches = re.findall(r'\bsound\b', description.lower())
sound_count = len(sound_matches)

# Check for forbidden words
has_forbidden = bool(re.search(r'\b(music|audio)\b', description, re.IGNORECASE))

# Check all constraints
issues = []
if sentence_count != 3:
    issues.append(f'Sentences: {sentence_count} (need exactly 3)')
if word_count < 25 or word_count > 30:
    issues.append(f'Words: {word_count} (need 25-30)')
# ...

return [{'json': {'all_pass': len(issues) == 0, 'issues': issues}}]
```

**Why a Python tool instead of an LLM?**
- **Faster:** No API call needed
- **Cheaper:** No tokens used
- **Accurate:** Code counts exactly right (LLMs often miscount)

**Key concept:** The agent doesn't "know" how to count — it *delegates* all validation to the tool. The LLM orchestrates; the code validates.

### Node-by-Node Walkthrough (V2)

| Node | Type | What it does |
|------|------|-------------|
| **Run: V2 Agent** | Manual Trigger | Starts Version 2 |
| **Input — Product (V2)** | Set | Creates `product` and `constraints` |
| **Reflection Agent (V2)** | AI Agent | Writes, validates, refines in a loop |
| **Validator Tool (Python)** | Python Code Tool | Python function that checks constraints |
| **Output (V2)** | Set | Returns `final_description` and `approach` |

**Sub-nodes connected to the Agent:**
- **OpenRouter (V2)** — the LLM brain
- **Validator Tool (Python)** — the constraint checker

### Prompt Used (V2)

**System Message:**
```
You are a product copywriter who writes precise descriptions.

Process:
1. Write a draft description
2. Use the Validator tool to check ALL constraints
3. If any constraint fails, revise and validate again
4. Repeat until all_pass is true
5. Return the final description

Rules:
- Count words carefully
- Call the Validator tool at most 5 times
- When all_pass is true, return ONLY the final description
```

Notice: the system message describes the reflection loop explicitly. The agent follows the same pattern we built manually in V1!

---

## The Insight: Agents Are Automated Loops

Compare what each version automates:

| Version | You design... | n8n handles... | AI handles... |
|---------|---------------|----------------|---------------|
| **V1: Loop** | The loop structure, exit condition | Looping, iteration count | Text generation |
| **V2: Agent** | The goal and tools | Everything | The entire loop |

**The agent node is not magic.** It's automating the same Generate → Critique → Refine loop you built by hand.

```
V1 Loop:        You ──────────▶ n8n ──────────────▶ LLM
                (design loop)   (runs loop)

V2 Agent:       You ──▶ n8n Agent Node ───────────▶ LLM
                (goal)  (runs loop internally)
```

```{note}
**Terminology: Evaluator-Optimizer.** Some sources call this the **Evaluator-Optimizer** pattern — a separate evaluator judges the output while an optimizer refines it. That is exactly what both versions do here: V1 uses a dedicated Validate LLM, V2 uses a Python tool. Pure "reflection" where the *same* LLM critiques its own output is a special case of this broader pattern.
```

---

## When to Use Reflection

Reflection adds latency and cost. Use it strategically:

| Use Reflection when... | Skip Reflection when... |
|------------------------|-------------------------|
| Constraints are strict and measurable | Task is simple or one-shot |
| First-pass accuracy is typically low | Real-time / low-latency required |
| Output quality matters more than speed | Constraints are fuzzy or subjective |
| You can define a clear validator | No clear "done" criteria exists |

### V1 vs V2: Which Approach?

| Situation | Best approach |
|-----------|---------------|
| You need full control over each step | V1: Loop |
| You want to debug exactly what's happening | V1: Loop |
| Task is ambiguous, you want AI to adapt | V2: Agent |
| You want the simplest implementation | V2: Agent |

**General rule:** Start with V2 (agent). If you need more control or debugging, build V1.

---

## Try It Yourself

1. **Run both versions** on the same product
2. **Compare:** How many iterations did each take? Which produced the best output?
3. **Change the constraints:** Make them harder (e.g., exactly 27 words) and run again
4. **Observe the agent:** In V2, click the agent node to see its internal tool calls

**Challenge:** Modify V1 to allow up to 10 iterations. Does it ever need that many?

---

## Key Takeaways

1. **Reflection = Generate → Critique → Refine → Repeat**
2. **You can build the loop manually** (V1) or **let the agent handle it** (V2)
3. **Agents are not magic** — they automate the same loop you can build by hand
4. **Always include guardrails** (max iterations) to prevent infinite loops
5. **Python tools are useful** when you need precise validation (LLMs miscount)

**Next:** In the **First AI Agent** chapter, we'll explore more agent capabilities — memory, multiple tools, and conversation.